<a href="https://colab.research.google.com/github/AvantiShri/colab_notebooks/blob/master/revcomp/ReplicateRevcompPaperResultsInKeras2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pull in the hdf5 files used for training
!wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Max/train_data.hdf5
!wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Max/valid_data.hdf5
!wget http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Max/test_data.hdf5

--2020-09-09 07:24:05--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Max/train_data.hdf5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203012944 (194M)
Saving to: ‘train_data.hdf5’

train_data.hdf5     100%[===================>] 193.61M  20.3MB/s    in 10s     

2020-09-09 07:24:16 (18.6 MB/s) - ‘train_data.hdf5’ saved [203012944/203012944]

--2020-09-09 07:24:16--  http://mitra.stanford.edu/kundaje/avanti/rc_data/backup_revcomppaperdata/Gm12878/Max/valid_data.hdf5
Resolving mitra.stanford.edu (mitra.stanford.edu)... 171.67.96.243
Connecting to mitra.stanford.edu (mitra.stanford.edu)|171.67.96.243|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13683729 (13M)
Saving to: ‘valid_data.hdf5’

valid_data.hdf5     100%[===================>]  13.05M  8.59MB/s    in 1.5s   

In [2]:
!pip install momma_dragonn

  Created wheel for momma-dragonn: filename=momma_dragonn-0.2.7.0-cp36-none-any.whl size=35041 sha256=e7c633f338fef4d76e1d968f91493d100d31a004e5da782a98775375c625b17c
  Stored in directory: /root/.cache/pip/wheels/87/e8/25/a7325bcfdc08dc594687cc1cda8915c217c602fa5586e4835f
  Created wheel for avutils: filename=avutils-0.2.2.0-cp36-none-any.whl size=18627 sha256=9e047c9b972bbb5ad09a39d6a11a488f1ad164690601e87f498de1c0578fa569
  Stored in directory: /root/.cache/pip/wheels/c2/10/60/224c0029355b3627d218534df8cbf15588409cdd6bc8df0993
Successfully built momma-dragonn avutils


In [3]:
import momma_dragonn

def get_train_data_loader():
  return momma_dragonn.data_loaders.hdf5_data_loader.MultimodalBatchDataLoader(
                          path_to_hdf5="train_data.hdf5",
                          bundle_x_and_y_in_generator=False,
                          strip_enclosing_dictionary=True,
                          batch_size=100,
                          num_to_load_for_eval=1000)

valid_data_loader = momma_dragonn.data_loaders.hdf5_data_loader.MultimodalAtOnceDataLoader(
    path_to_hdf5="valid_data.hdf5", strip_enclosing_dictionary=True)
valid_data = valid_data_loader.get_data()

/usr/local/lib/python3.6/dist-packages/momma_dragonn/data_loaders/hdf5_data_loader.py:32: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.f = h5py.File(self.path_to_hdf5)


Input modes ['sequence']
Output modes ['output']


In [4]:
test_data_loader = momma_dragonn.data_loaders.hdf5_data_loader.MultimodalAtOnceDataLoader(
    path_to_hdf5="test_data.hdf5", strip_enclosing_dictionary=True)
test_data = test_data_loader.get_data()

/usr/local/lib/python3.6/dist-packages/momma_dragonn/data_loaders/hdf5_data_loader.py:32: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.f = h5py.File(self.path_to_hdf5)


Input modes ['sequence']
Output modes ['output']


In [5]:
!pip install keras-genomics

  Created wheel for keras-genomics: filename=keras_genomics-0.1.2.1-cp36-none-any.whl size=10699 sha256=23afdf912294f6802b7f1d3623d131b79b2c7c3ee25e35d690b201e90944efe0
  Stored in directory: /root/.cache/pip/wheels/2a/8e/eb/17f924ed090e1af0773d925dadd42339639e1b87c8ff17dff8
Successfully built keras-genomics


In [6]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [7]:
import keras
import tensorflow as tf
import numpy as np
import keras
from sklearn.metrics import roc_auc_score


def set_seed(seed):
  np.random.seed(seed)
  tf.set_random_seed(seed)


#define the architectures
def standard_architecture():
  model = keras.models.Sequential()
  model.add(keras.layers.Convolution1D(
              input_shape=(1000,4), nb_filter=16, filter_length=15))
  model.add(keras.layers.normalization.BatchNormalization())
  model.add(keras.layers.core.Activation("relu"))
  model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
  model.add(keras.layers.normalization.BatchNormalization())
  model.add(keras.layers.Activation("relu"))
  model.add(keras.layers.convolutional.Convolution1D(
            nb_filter=16, filter_length=14))
  model.add(keras.layers.normalization.BatchNormalization())
  model.add(keras.layers.Activation("relu"))
  model.add(keras.layers.convolutional.MaxPooling1D(pool_length=40, stride=20))
  model.add(keras.layers.core.Flatten())
  model.add(keras.layers.core.Dense(output_dim=1, trainable=True,
                                    init="glorot_uniform"))
  model.add(keras.layers.core.Activation("sigmoid"))
  return model


class AuRocCallback(keras.callbacks.Callback):

  def __init__(self, model, valid_X, valid_Y):
    self.model = model
    self.valid_X = valid_X
    self.valid_Y = valid_Y
    self.best_auroc_sofar = 0.0
    self.best_weights = None
    self.best_epoch_number = 0

  def on_epoch_end(self, epoch, logs):
    preds = self.model.predict(self.valid_X)
    auroc = roc_auc_score(y_true=self.valid_Y, y_score=preds)
    print("Epoch",epoch,"auroc:",auroc)
    if (auroc > self.best_auroc_sofar):
      print("New best auroc")
      self.best_weights = self.model.get_weights()
      self.best_epoch_number = epoch
    

def train_model(model_creator, seed, train_data_loader, valid_data):
  set_seed(seed=seed)
  model = model_creator()
  model.compile(loss="binary_crossentropy", metrics=["accuracy"],
                optimizer=keras.optimizers.Adam(lr=0.001))

  EPOCHS_TO_TRAIN_FOR=80
  early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience=EPOCHS_TO_TRAIN_FOR,
                              restore_best_weights=True)
  auroc_callback = AuRocCallback(model=model,
                                 valid_X=valid_data.X,
                                 valid_Y=valid_data.Y)
  history = model.fit_generator(
      train_data_loader.get_batch_generator(),
      validation_data=(valid_data.X, valid_data.Y),
      epochs=EPOCHS_TO_TRAIN_FOR,
      steps_per_epoch=50,
      class_weight={0:1, 1: 16.5},
      callbacks=[auroc_callback,
                 early_stopping_callback],
      
  )
  print("best epoch:", early_stopping_callback.best)
  model.set_weights(early_stopping_callback.best_weights)
  print("Validation set AUROC with best-loss early stopping:",
        roc_auc_score(y_true=valid_data.Y, y_score=model.predict(valid_data.X)))
  print("Test set AUROC with best-loss early stopping:",
        roc_auc_score(y_true=test_data.Y, y_score=model.predict(test_data.X)))
  model.set_weights(auroc_callback.best_weights)
  print("Validation AUROC at best-auroc early stopping:",
        roc_auc_score(y_true=valid_data.Y, y_score=model.predict(valid_data.X)))
  print("Validation AUROC at best-auroc early stopping:",
        roc_auc_score(y_true=test_data.Y, y_score=model.predict(test_data.X)))

train_model(model_creator=standard_architecture,
            seed=0, train_data_loader=get_train_data_loader(),
            valid_data=valid_data)

Using TensorFlow backend.


Input modes ['sequence']
Output modes ['output']
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


/usr/local/lib/python3.6/dist-packages/momma_dragonn/data_loaders/hdf5_data_loader.py:12: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.f = h5py.File(self.path_to_hdf5)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:21: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(filters=16, kernel_size=14)`



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:28: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=40, strides=20)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(trainable=True, units=1, kernel_initializer="glorot_uniform")`



Epoch 1/80
50/50 [==============================] - 15s 296ms/step - loss: 1.1058 - accuracy: 0.7016 - val_loss: 0.7167 - val_accuracy: 0.4739
Epoch 0 auroc: 0.8245684143222506
New best auroc
Epoch 2/80
50/50 [==============================] - 7s 136ms/step - loss: 0.8939 - accuracy: 0.7996 - val_loss: 0.8526 - val_accuracy: 0.3156
Epoch 1 auroc: 0.8442595065836754
New best auroc
Epoch 3/80
50/50 [==============================] - 7s 138ms/step - loss: 0.9592 - accuracy: 0.7702 - val_loss: 0.8960 - val_accuracy: 0.2913
Epoch 2 auroc: 0.8398857731498397
New best auroc
Epoch 4/80
50/50 [==============================] - 7s 136ms/step - loss: 0.8670 - accuracy: 0.8124 - val_loss: 0.9280 - val_accuracy: 0.4409
Epoch 3 auroc: 0.8524272468486024
New best auroc
Epoch 5/80
50/50 [==============================] - 7s 138ms/step - loss: 0.9761 - accuracy: 0.7812 - val_loss: 0.5773 - val_accuracy: 0.7296
Epoch 4 auroc: 0.8518329482204163
New best auroc
Epoch 6/80
50/50 [=========================

In [8]:
import keras_genomics


#define the revcomp architectures
def revcomp_architecture():
  model = keras.models.Sequential()
  model.add(keras_genomics.layers.convolutional.RevCompConv1D(
              input_shape=(1000,4), nb_filter=16, filter_length=15))
  model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
  model.add(keras.layers.core.Activation("relu"))
  model.add(keras_genomics.layers.convolutional.RevCompConv1D(
            nb_filter=16, filter_length=14))
  model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
  model.add(keras.layers.Activation("relu"))
  model.add(keras_genomics.layers.convolutional.RevCompConv1D(
            nb_filter=16, filter_length=14))
  model.add(keras_genomics.layers.normalization.RevCompConv1DBatchNorm())
  model.add(keras.layers.Activation("relu"))
  model.add(keras.layers.convolutional.MaxPooling1D(pool_length=40, stride=20))
  model.add(keras_genomics.layers.convolutional.WeightedSum1D(
              symmetric=False, input_is_revcomp_conv=True))
  model.add(keras.layers.core.Dense(output_dim=1, trainable=False,
                                    init="ones"))
  model.add(keras.layers.core.Activation("sigmoid"))
  return model


train_data_loader = momma_dragonn.data_loaders.hdf5_data_loader.MultimodalBatchDataLoader(
                          path_to_hdf5="train_data.hdf5",
                          bundle_x_and_y_in_generator=False,
                          strip_enclosing_dictionary=True,
                          batch_size=100,
                          num_to_load_for_eval=1000)
train_model(model_creator=revcomp_architecture,
            seed=0, train_data_loader=get_train_data_loader(),
            valid_data=valid_data)

Input modes ['sequence']
Output modes ['output']
Input modes ['sequence']
Output modes ['output']


/usr/local/lib/python3.6/dist-packages/momma_dragonn/data_loaders/hdf5_data_loader.py:12: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  self.f = h5py.File(self.path_to_hdf5)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(input_shape=(1000, 4), filters=16, kernel_size=15)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `RevCompConv1D` call to the Keras 2 API: `RevCompConv1D(filters=16, kernel_size=14)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `RevCompConv1D` call to the Ke

Epoch 1/80
50/50 [==============================] - 8s 168ms/step - loss: 1.4024 - accuracy: 0.5892 - val_loss: 0.3632 - val_accuracy: 0.8927
Epoch 0 auroc: 0.8289228098211473
New best auroc
Epoch 2/80
50/50 [==============================] - 7s 140ms/step - loss: 0.8487 - accuracy: 0.8154 - val_loss: 0.3340 - val_accuracy: 0.8754
Epoch 1 auroc: 0.8574631674056228
New best auroc
Epoch 3/80
50/50 [==============================] - 7s 142ms/step - loss: 0.9057 - accuracy: 0.7960 - val_loss: 0.4954 - val_accuracy: 0.8082
Epoch 2 auroc: 0.8603006321996092
New best auroc
Epoch 4/80
50/50 [==============================] - 7s 139ms/step - loss: 0.8072 - accuracy: 0.8416 - val_loss: 0.4092 - val_accuracy: 0.8484
Epoch 3 auroc: 0.8687925914588319
New best auroc
Epoch 5/80
50/50 [==============================] - 7s 142ms/step - loss: 0.8861 - accuracy: 0.8148 - val_loss: 0.3760 - val_accuracy: 0.8669
Epoch 4 auroc: 0.869631117233419
New best auroc
Epoch 6/80
50/50 [============================